In [7]:
import os 
import json
import cv2 as cv
import numpy as np
import matplotlib.pyplot as plt

In [8]:
fily = json.load(open('testing.json'))

In [9]:
def vid_gen(fily: dict) -> dict:
    """generates a dict where {key: value} is {vid: filename}"""
    vid = {}
    for key, value in fily['file'].items():
        vid[key] = value['fname']
    return vid

def vid_tr_gen(vid: dict) -> dict:
    """generates a dict where {key: value} is {filename: vid}"""
    vid_tran = {}
    for key, value in vid.items():
        vid_tran[value] = key
    return vid_tran
def label_gen(vid: dict, fily: dict)-> dict:
    """generates a dict where {key: value} is {vid: [list of label names]}"""
    labels = {}
    for key, value in vid.items():
        labels[key] = []
    for key, value in fily['metadata'].items():
        if len(fily['metadata'][key]['xy']) != 0:
            labels[value['vid']] += [key]
    return labels

vid = vid_gen(fily)
vid_tran = vid_tr_gen(vid)
labels = label_gen(vid, fily)

In [10]:
vid

{'1': '2.jpg',
 '2': '46.jpg',
 '3': '70.jpg',
 '4': '74.jpg',
 '5': '139.jpg',
 '6': '169.jpg',
 '7': '202.jpg',
 '8': '230.jpg',
 '9': '276.jpg',
 '10': '369.jpg'}

In [11]:
labels

{'1': ['3_yLqmtfJd',
  '3_X5AysGcg',
  '3_9k2ozjGZ',
  '3_klNseqZC',
  '3_A03YRxB8',
  '3_MSApcdgw',
  '3_8hVyCF6K',
  '3_JTKTXXWG',
  '3_h0Bld1Jk',
  '3_mBoNGOGp',
  '3_XmMOJyze',
  '3_jZc7Tyt6',
  '3_fOH5ItSi',
  '3_X3wrVsW4',
  '3_lJSNkiAa',
  '3_JVdReZIt',
  '3_2zJSjirL',
  '3_MWs75hGu',
  '3_1OCI2kte',
  '3_lQ49COZW',
  '3_J2bXuxdj'],
 '2': ['47_eCSEvO4K',
  '47_NucOowmW',
  '47_8JLX6EOE',
  '47_g7EFYgIp',
  '47_N45CLXMy',
  '47_UsIrVFKb',
  '47_jjvgs9BR',
  '47_IXNZWVMY',
  '47_8i9FhQkr',
  '47_7mdxmKSJ',
  '47_vSO9UM59',
  '47_pielBL35',
  '47_wayUBMYX',
  '47_ndb0DQnr',
  '47_gOmCtTrK',
  '47_1NRUSfU6',
  '47_L2W3ZNEX'],
 '3': ['71_8TCEh9zs', '71_z9V94uqd', '71_m9s2nGcu', '71_PZRCqzhb'],
 '4': ['75_92NG4rxZ', '75_KwAnMnni', '75_DQ2CppYh'],
 '5': ['140_TlTOJijp'],
 '6': ['170_5tlzmhRZ'],
 '7': ['203_bg9zHYDn',
  '203_WuVNlLFX',
  '203_13wtSZTB',
  '203_enNvBxq1',
  '203_5XzyldqF',
  '203_kDpcbpJB',
  '203_Wu1DXPfD',
  '203_zwEdPWfG',
  '203_scaZbtX0',
  '203_DBcJG1xd',
  '203_yd

In [12]:
for key, value in labels.items():
    mask_dir = 'segmentation/masks/'+ f'{vid[key].split('.')[0]}'
    img_dir = 'segmentation/samples/'+ f'{vid[key].split('.')[0]}'
    img = cv.imread('images/'+f"{vid[key]}")
    for k in range(len(value)):
        label = value[k]
        target = fily['metadata'][label]['xy']
        contours = []
        x_max = -1
        y_max = -1
        x_min = 5000
        y_min = 5000
        for i in range(1, len(target), 2):
            x = target[i]
            y = target[i+1]
            x_max = max(x_max, x)
            y_max = max(y_max, y)
            x_min = min(x_min, x)
            y_min = min(y_min, y)
            contours.append([x, y])
        contours = np.array(contours)
        contours -= [x_min, y_min]
        mask = np.zeros([y_max - y_min,x_max - x_min])
        cv.drawContours(mask, [contours], -1,255, -1)
        cv.imwrite(mask_dir + f"_{k}.jpg", mask)
        cv.imwrite(img_dir + f"_{k}.jpg", img[y_min:y_max, x_min:x_max])

error: OpenCV(4.10.0) /io/opencv/modules/imgcodecs/src/loadsave.cpp:798: error: (-215:Assertion failed) !_img.empty() in function 'imwrite'
